In [1]:
pip install datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=48e91599623b0756f253fe6ff848d49c0e9b637a5eb7e9b82f5ec2a8be45eae8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempt

In [2]:
import torch

In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import datasets
import transformers
from datasets import Dataset
import json
import torch


In [5]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer, AutoModelForQuestionAnswering

# Use model
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")
model.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [6]:

def load_and_preprocess_squad(input_file):
  with open(input_file, 'r', encoding='utf-8') as f:
      data = json.load(f)

  contexts = []
  questions = []
  answers = []
  for dataJson in data:
    for article in dataJson['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text'] if qa['answers'] else None
                answerStart = qa['answers'][0]['answer_start'] if qa['answers'] else None
                if answer is None:
                  print(question)
                if answer is None:
                  print(question)
                if answer:
                    contexts.append(context)
                    questions.append(question)
                    if not isinstance(answerStart, (int, float)):
                      answerStart = 0
                    answers.append({
                      "text": [answer.lower()],
                      "start": [answerStart]
                    })

  # Kiểm tra độ dài của các cột
  assert len(contexts) == len(questions) == len(answers)

  # Tạo từ điển dữ liệu
  dataset = {
      'context': contexts,
      'question': questions,
      'answer': answers
  }

  return dataset

In [32]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=1024,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["start"][0]
        end_char = answer["start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [33]:
dataset = load_and_preprocess_squad("/content/data/squad_2.0.json")
data_val = load_and_preprocess_squad("/content/data/eval_gd.json")

In [34]:
dataset_train = Dataset.from_dict(dataset)
dataset_eval = Dataset.from_dict(data_val)

In [26]:
dataset_train

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 2540
})

In [35]:
tokenized_squad = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)
tokenized_squad_eval = dataset_eval.map(preprocess_function, batched=True, remove_columns=dataset_eval.column_names)

Map:   0%|          | 0/2540 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [ ]:
tokenized_squad

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2950
})

In [36]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [37]:
training_args = TrainingArguments(
    output_dir="phobert_law",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=80,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad,
    eval_dataset=tokenized_squad_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.639318
2,No log,1.187644
3,2.125100,0.914440
4,2.125100,0.690556
5,2.125100,0.549003
6,1.199300,0.435856
7,1.199300,0.336014
8,1.199300,0.272215
9,0.790200,0.260047
10,0.790200,0.194719


TrainOutput(global_step=14800, training_loss=0.25377214012919247, metrics={'train_runtime': 6990.7521, 'train_samples_per_second': 33.759, 'train_steps_per_second': 2.117, 'total_flos': 1.5416508647424e+16, 'train_loss': 0.25377214012919247, 'epoch': 80.0})

In [ ]:
path = "/content/drive/MyDrive/transformers_model1"



In [ ]:
model.save_pretrained(path)
tokenizer.save_pretrained(path)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)

In [ ]:
trainer.save_model(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/transformers_model1/tokenizer_config.json',
 '/content/drive/MyDrive/transformers_model1/special_tokens_map.json',
 '/content/drive/MyDrive/transformers_model1/vocab.json',
 '/content/drive/MyDrive/transformers_model1/merges.txt',
 '/content/drive/MyDrive/transformers_model1/added_tokens.json',
 '/content/drive/MyDrive/transformers_model1/tokenizer.json')

In [ ]:
trainer.save_model()

JSONDecodeError: Expecting ',' delimiter: line 6 column 20 (char 69)